In [27]:
#  This code just reads the main pendency file 

import pandas as pd
from datetime import datetime
import pytz

time_zone = pytz.timezone('Asia/Kolkata')
current_time= datetime.now(time_zone)

df = pd.read_csv("Pendency_automation_report_ageing_greater_than_12.csv")
df


,bag_facility_source_name,bag_facility_current_name,bag_facility_destination_name,bag_type_ph,tracking_id_ekart,bag_id,bag_status,fact_updated_at,_source,bag_xdock,fact_updated_at_1
0,DehradunHub_DED_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3246255307,PH-ZO-DELYKB-30542756,REACHED,2023-11-18T19:49:42.000+05:30,BAG,True,2023-11-18T19:49:42.000+05:30
1,PanipatHub_PPT_PL,MotherHub_YKB,MotherHub_YKB,ZO,FMPC3241657280,PH-ZO-DELYKB-30317153,OPENED,2023-11-17T21:44:24.000+05:30,BAG,False,2023-11-17T21:44:24.000+05:30
2,KaithalHub_KTL_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3245290134,PH-ZO-DELYKB-30544056,REACHED,2023-11-19T01:14:03.000+05:30,BAG,True,2023-11-19T01:14:03.000+05:30
3,RohtakHub_ROH_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3248540465,PH-ZO-DELYKB-30481388,REACHED,2023-11-19T00:38:34.000+05:30,BAG,True,2023-11-19T00:38:34.000+05:30
4,HaldwaniHub_HWI_PL,MotherHub_BNL,MotherHub_YKB,B5,MYEP1007053879,PH-B5-DELYKB-30546073,REACHED,2023-11-18T23:48:32.000+05:30,BAG,True,2023-11-18T23:48:32.000+05:30
...,...,...,...,...,...,...,...,...,...,...,...
5133,RohtakHub_ROH_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3248374212,PH-ZO-DELYKB-30580149,REACHED,2023-11-19T00:33:23.000+05:30,BAG,True,2023-11-19T00:33:23.000+05:30
5134,RohtakHub_ROH_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3248384717,PH-ZO-DELYKB-30580149,REACHED,2023-11-19T00:33:23.000+05:30,BAG,True,2023-11-19T00:33:23.000+05:30
5135,RohtakHub_ROH_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3249330554,PH-ZO-DELYKB-30580149,REACHED,2023-11-19T00:33:23.000+05:30,BAG,True,2023-11-19T00:33:23.000+05:30
5136,RohtakHub_ROH_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPP2016327853,PH-ZO-DELYKB-30580149,REACHED,2023-11-19T00:33:23.000+05:30,BAG,True,2023-11-19T00:33:23.000+05:30


In [28]:
df = df[df['bag_facility_current_name'] == "MotherHub_YKB"]
df_zone_wise = df.groupby('bag_type_ph').size()
df_zone_wise

bag_type_ph
B5    381
ZO    597
dtype: int64

In [29]:

# Segregated ZO PH & SPH in this Cell

df_zo = df[df['bag_type_ph'] == "ZO"]
df_zo
df_zo_ph = df_zo[df_zo["bag_facility_source_name"].str.contains("DEL_PL|Bilaspur|Bhiwani")]
df_zo_ph_wise = df_zo_ph.groupby('bag_facility_source_name').size()
print(sum(df_zo_ph_wise))


df_zo_sph = df_zo[~df_zo.index.isin(df_zo_ph.index)]
df_zo_sph_wise = df_zo_sph.groupby('bag_facility_source_name').size()
print(sum(df_zo_sph_wise))



391
206


In [30]:
# Segregated B5 PH & SPH in this cell same as above

df_b5 = df[df['bag_type_ph'] == "B5"]
df_b5

df_b5_ph = df_b5[df_b5['bag_facility_source_name'].str.contains("DEL_PL|Bilaspur|Bhiwani")]
df_b5_ph_wise  = df_b5_ph.groupby('bag_facility_source_name').size()
print(sum(df_b5_ph_wise))


df_b5_sph  = df_b5[~df_b5.index.isin(df_b5_ph.index)]
df_b5_sph_wise = df_b5_sph.groupby('bag_facility_source_name').size()
print(sum(df_b5_sph_wise))

275
106


In [37]:
def calculate_and_categorize_time(dataframe, column_name, current_time  , bucket_name):
    df_time_wise = pd.to_datetime(dataframe[column_name])
    df_hour_wise = (current_time - df_time_wise) / pd.Timedelta(hours=1)
    df_hour_wise = df_hour_wise.round().astype(int)
    
    def categorize_time(hour_diff):
        if hour_diff > 48:
            return '> 48 hours'
        elif hour_diff > 24 and hour_diff < 48:
            return '24-48 hours'
        elif hour_diff > 12:
            return '12-24 hours'
        else:
            return '< 12 hours'
    
    dataframe["hour_diff"] = df_hour_wise
    dataframe[bucket_name] = df_hour_wise.apply(categorize_time)
    data = dataframe[bucket_name].value_counts()
    return data 


final_zo_ph = calculate_and_categorize_time(df_zo_ph , 'fact_updated_at' , current_time , 'ZO PH')
# print(f"{final_zo_ph} & total {sum(final_zo_ph)}" )
print(final_zo_ph)
a = final_zo_ph

final_zo_ph.to_csv("a.csv")
print(a)
print(type(a))



# final_zo_sph = calculate_and_categorize_time(df_zo_sph , 'fact_updated_at' , current_time , "ZO SPH")
# print(f"{final_zo_sph}  && Total:  {sum(final_zo_sph)}")
# print(f"ZO Total PH + SPH =   {sum(final_zo_sph) + sum(final_zo_ph)}")
# final_b5_ph = calculate_and_categorize_time(df_b5_ph , 'fact_updated_at' , current_time , "B5 PH")
# print(f" {final_b5_ph} && Total:  {sum(final_b5_ph)}")
# final_b5_sph = calculate_and_categorize_time(df_b5_sph , 'fact_updated_at' , current_time , "B5 SPH")
# print(f"{final_b5_sph} && Total {sum(final_b5_sph)}")
# print(f"B5 Final PH + SPH =  {sum(final_b5_ph) + sum(final_b5_sph)}")

ZO PH
> 48 hours     340
24-48 hours     48
12-24 hours      3
Name: count, dtype: int64
ZO PH
> 48 hours     340
24-48 hours     48
12-24 hours      3
Name: count, dtype: int64
<class 'pandas.core.series.Series'>


C:\Users\abhishek.h1\AppData\Local\Temp\ipykernel_18600\3200558967.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["hour_diff"] = df_hour_wise
C:\Users\abhishek.h1\AppData\Local\Temp\ipykernel_18600\3200558967.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[bucket_name] = df_hour_wise.apply(categorize_time)
